# EEG Classification

#Import packages

In [2]:
from glob import glob
import os
import mne
import numpy as np
import pandas
import matplotlib.pyplot as plt

#Import the files and create a variable

In [3]:
all_file_path = glob('dataverse_files/*.edf')
print(len(all_file_path))

28


In [4]:
all_file_path[0]

'dataverse_files\\h01.edf'

#Separate healthy and shizophrenia patients

In [5]:
healthy_file_path=[i for i in all_file_path if 'h' in i.split('\\')[1]]
patient_file_path=[i for i in all_file_path if 's' in i.split('\\')[1]]
print(len(healthy_file_path),len(patient_file_path))

14 14


#Filter and segment data

In [6]:
def read_data(file_path):
    data=mne.io.read_raw_edf(file_path,preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    array=epochs.get_data()
    return array

In [7]:
sample_data=read_data(healthy_file_path[0])

Extracting EDF parameters from C:\Users\DIPANKAR KARMAKAR\dataverse_files\h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1651 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Not setting metadata
231 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 231 events and 1250 original time points ...


[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    0.6s finished


0 bad epochs dropped


In [8]:
sample_data.shape # number of epochs, channels, length of signal

(231, 19, 1250)

In [9]:
%%capture
control_epochs_array=[read_data(i) for i in healthy_file_path]
patient_epochs_array=[read_data(i) for i in patient_file_path]

In [10]:
control_epochs_array[0].shape,control_epochs_array[1].shape

((231, 19, 1250), (227, 19, 1250))

In [11]:
control_epoch_labels=[len(i)*[0] for i in control_epochs_array]
patient_epoch_labels=[len(i)*[0] for i in patient_epochs_array]
len(control_epoch_labels),len(patient_epoch_labels)

(14, 14)

In [12]:
data_list=control_epochs_array+patient_epochs_array
label_list=control_epoch_labels+patient_epoch_labels

In [13]:
group_list=[[i]*len(j) for i,j in enumerate(data_list)]
len(group_list)

28

In [14]:
data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(group_list)
print(data_array.shape,label_array.shape,group_array.shape)

(7201, 19, 1250) (7201,) (7201,)
